### Using the Home Assistant Python API

In [2]:
import homeassistant.remote as remote

In [3]:
api = remote.API('127.0.0.1', 'YOUR_PASSWORD')

In [8]:
print(remote.validate_api(api))

ok


In [9]:
office_temperature = remote.get_state(api, 'sensor.kitchen_temperature')

In [10]:
print('{} is {} {}.'.format(office_temperature.attributes['friendly_name'],
                            office_temperature.state,
                            office_temperature.attributes['unit_of_measurement']
                            )
      )

Kitchen Temperature is 23 °C.
